# Week 3 Assignment


## Part 1
#### In this part, I will extracting Toronto Neighborhoods information from wiki page and store the information into a dataframe

In [2]:
import requests
import pandas as pd
# !conda install -c anaconda beautifulsoup4 --yes  # uncomment this message if beautiful is not installed
from bs4 import BeautifulSoup

Requesting the information from the website and read it's contents using Beautiful Soup library

In [3]:
webpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = BeautifulSoup(webpage.text, 'html.parser')
#contents

By looking at the contents, we found that the information we want are stored under the table attribute, so we find the table using the find method

In [4]:
table = contents.find('table', attrs={'class':'wikitable sortable'})
#table

Looking at the table, we find every entry is stored between the tr tag, and each element in the entry are stored under td tag, so we need to loop through the table fo find all the information. But before that, we need to create an empty dataframe first used to store the information

In [5]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
df

,Postalcode,Borough,Neighborhood


In the following loop, we loop through each entry and each element in the entry, notice that the second element in each entry, which is the 'Borough' column, can't be 'Not assigned', so we filter that out, also, if the 'Borough' is assigned but the 'Neighborhood' is 'Not assigned' we assign the Borough value to the Neighborhood value

In [6]:
for tr_tag in table.find_all('tr'):
    row = []
    for td_tag in tr_tag.find_all('td'):
        row.append(td_tag.text.strip())
    if len(row)==3 and row[1] != 'Not assigned':
        if row[2] == 'Not assigned':
            row[2] = row[1]
        df.loc[len(df)] = row

In [7]:
df.head(11)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Next, we create an temp df which we store the information where we group the dataframe by post code and combine the neighborhood in that post code

In [8]:
tmp_df = df.groupby(['Postalcode'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).to_frame()
tmp_df.head()

,Neighborhood
Postalcode,
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae


In [9]:
tmp_df.reset_index(inplace=True)
tmp_df.head()

,Postalcode,Neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


Merge the old dataframe with the new one with the postalcode column, drop the Neighborhood column in the old dataframe and rename the new colomn and we are done!

In [10]:
df_merge = pd.merge(df, tmp_df, on='Postalcode')
df_merge.head()

,Postalcode,Borough,Neighborhood_x,Neighborhood_y
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,Harbourfront
3,M6A,North York,Lawrence Heights,"Lawrence Heights, Lawrence Manor"
4,M6A,North York,Lawrence Manor,"Lawrence Heights, Lawrence Manor"


In [11]:
df_merge.drop(['Neighborhood_x'], axis=1, inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_y':'Neighborhood'},inplace=True)
df_merge.shape

(103, 3)

In [14]:
df_merge.head()


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [15]:
df_merge.shape

(103, 3)

## Part 2
#### In this part, we will upload a csv file, which containing the geographical coordinates of each postal code in Toronto area, and add this coordinates information into the previous data frame

In [16]:
df_coor = pd.read_csv('http://cocl.us/Geospatial_data')

In [17]:
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We found that the column name is different from the previous dataframe, so we change the name of the column

In [18]:
df_coor.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

In [20]:
df_coor.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two dataframes

In [21]:
df_coor_merged = pd.merge(df_coor, df_merge, on='Postalcode')

In [22]:
df_coor_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


Move the column Latitude and Longitude to the end of the dataframe

In [23]:
neighborhood = df_coor_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [24]:
neighborhood.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [25]:
neighborhood.shape

(103, 5)